# Participants:
Marta Almagro Fuello: 100451979

Gracia Estrán Buyo: 100452014

# Comparision between classical Matrix Multiplication Algorithm and Tiled Matrix Multiplication

In this practical work we will compare these two algorithms, and check how the memory access impacts in the performance.

These 2 cells will help us to know which hardware have assigned, to decide the block size in iur parallel algorithm.

Sometimes the 256 threads per block (16x16 threads) are the bes compromise solution.

In [3]:
#Uncomment the follow line if you are running in Google Colaboratory
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 7.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 70 kB 8.6 MB/s 
  Created wheel for pycuda: filename=pycuda-2022.1-cp37-cp37m-linux_x86_64.whl size=629701 sha256=75df20950bc2f24dd8a216b3d9970e833eb86c34bfa8b9e98e64aac01285f65e
  Stored in directory: /root/.cache/pip/wheels/17/53/c9/caa05618e686df51f017d8a9923f38d915ce31df67ab6628e6
  Created wheel for pytools: filename=pytools-2022.1.12-py2.py3-none-any.whl size=65034 sha256=875c238407558d7533f040651be7dece6b512d2374a88d86bf3d697377943f2b
  Stored in directory: /root/.cache/pip/wheels/37/5e/9e/76d7430e116b7cab0016fbabb26b896daae1946a3f7dea9915
Successfully built pycuda pytools


Import the necessary modules, as pycuda, numpy and time

In [4]:
import  numpy  as  np
import  pycuda.autoinit
from    pycuda.compiler import SourceModule
import  pycuda.driver as  drv
import  pycuda.gpuarray as  gpuarray
import  pycuda.tools as gtools
from numpy import linalg as la
import time

# FIRST KERNEL: simple matrices multiplication
This kernel will recive 3 matrices: a and b are the source matrices, and c is where we will store the result.

The size of the matrix a is $m *n$.

The size of the matrix b is $n * o$.

The size of the matrix c is $m * o$.

All three matrices are stored in a row-wise vector. 

Each thread will have assigned one cell position in the matrix c. The formulae for both coordinates are:


$idxY = blockIdx.y*blockDim.y+threadIdx.y$

$idxX = blockIdx.x*blockDim.x+threadIdx.x$



In [5]:
kernel  =  SourceModule ("""
__global__ void matrix_mult(float* a, 
                            float* b, 
                            float* c, 
                            int m, 
                            int n, 
                            int o) 
{ 
  // a is the vector which represents the matrix_a
  // b is the vector which represents the matrix_b
  // c is the vector where we will store the resulting matrix
  // m is number of rows of matrix a
  // n is the number of columns of matrix a, and number of rows of matrix b
  // o is the number of columns of matrix b
  // First task: Using threadIdx.x, threadIdx.y, blockDim.x, blockDim.y, 
  // blockIdx.x, blockDim.y identify the coordinates x and y in the result matrix
  // implements the matrix multiplication, cell by cell using the conventional code and analyze

  int idxX;
  int idxY;
  int idxZ;
  int offA;

  float s;

  idxY = blockIdx.y*blockDim.y+threadIdx.y; //With this we calculate the row address in target matrix
  idxX = blockIdx.x*blockDim.x+threadIdx.x; //Here we calculate the column address in target matrix


  if ( idxX < o && idxY < m ){    //Here we verify the row address and column address are valid
    idxZ = idxY*o + idxX;         //Here we calculate the target vector address, 
                                  //asuming it is a row wise matrix representation
    s = 0;                        //Initialize the s acumulator

    offA = idxY*n;                //We calculate the offset of row in matrix a row wise representation
                                  //This is to reduce the number of calculae in the next for

    for ( int i =0; i<n; i++)     //Here we run through the a columns, b rows
      s += a[offA+i]*b[(i*o)+idxX];
    
    c[idxZ]=s;
  }
  
}
""")

We fetch the cuda function matrix_mult and assign a python reference.

In [6]:
matrix_mult= kernel.get_function("matrix_mult")

For the first case, we creates 3 matrices of size 1024 * 1024 (this is for a fair play comparation with the tiled matrix multiplication).

In [7]:
numRowsA=1024
numColsA=1024
numRowsB=1024
numColsB=1024

In [8]:
matrix_a=np.random.randn(numRowsA,numColsA).astype(np.float32)
matrix_b=np.random.randn(numRowsB,numColsB).astype(np.float32)
matrix_c=np.zeros((numRowsA,numColsB),dtype=np.float32)

Here, we upload the matrices to the GPU.

In [9]:
matrix_a_gpu=gpuarray.to_gpu(matrix_a)
matrix_b_gpu=gpuarray.to_gpu(matrix_b)
matrix_c_gpu=gpuarray.to_gpu(matrix_c)

We define a block execution size of 16x16x1 threads. It will allocates 256 threads per block < 1024 which is the most common maximum threads per block.

Additionally, Most GPUs have a multiple core number of 256. This allow us to have the maximum number of blox in execution.

In [10]:
block_size=(16,16,1) #We take this value to get up to 256 parallel threads per block 
                     #This will allow us to get up to 9 parallel blocks in execution in a K80
                     #10 parallel blocks in execution in a T4
                     #14 parallel blocks in execition in a GTX 1080 Ti
                     #17 parallel blocks in execition in a RTX 2080 Ti

We calculate the number of blocks in x axis we will need, as well the number of blocks in y axis..

In [11]:
numblocks_x=int(np.ceil(numColsB/16))

In [12]:
numblocks_y=int(np.ceil(numRowsA/16))

Here, we define the grid size

In [13]:
grid_size=(numblocks_x,numblocks_y)

And execute our matrices multiplication algorithm.

In [14]:
start_t = time.time()
matrix_mult(matrix_a_gpu,
            matrix_b_gpu,
            matrix_c_gpu,
            np.int32(numRowsA),
            np.int32(numColsA),
            np.int32(numColsB),
            block=block_size,
            grid=(numblocks_x,numblocks_y))
end_t = time.time()

Calculates the time it uses 

In [15]:
print("Time processing: {0} seconds".format(end_t-start_t))

Time processing: 0.0018801689147949219 seconds


And get the C result matrix, to compare with the locally computed result.

In [16]:
matrix_c_final=matrix_c_gpu.get()

In [17]:
d = np.matmul(matrix_a,matrix_b)

In [18]:
MSE = np.sum(np.sum(np.power(matrix_c_final-d,2)))/(d.shape[0]*d.shape[1])

In [19]:
print("Mean Square Error: {0}".format(MSE))

Mean Square Error: 3.4938357762470673e-10


In [20]:
la.norm(matrix_c_final-d)

0.019140251

In [21]:
matrix_c_final[290:299,90:99]

array([[-1.23125267e+01,  3.67383499e+01, -2.99956665e+01,
        -3.45916061e+01, -2.92059059e+01, -2.44092426e+01,
         3.27181745e+00,  4.72440529e+01,  3.31779327e+01],
       [ 7.32669830e+00,  5.68259321e-02, -4.18285275e+00,
        -2.37644863e+01, -3.24902611e+01,  3.21666946e+01,
        -1.20793664e+00, -9.60221958e+00,  2.28222618e+01],
       [-2.84691715e+01, -2.13265114e+01,  1.03802233e+01,
        -4.61409903e+00, -2.47352982e+01,  8.26731110e+00,
         1.00101366e+01, -7.72007799e+00, -5.75500345e+00],
       [ 9.77897358e+00, -3.21664505e+01,  1.43078213e+01,
        -3.78362617e+01,  2.56115913e+00, -6.56047440e+01,
         5.96568632e+00,  8.74001884e+00,  5.66848259e+01],
       [-4.50232201e+01,  1.36028280e+01, -4.12081337e+01,
        -3.52544937e+01, -1.31826725e+01, -1.00145073e+01,
         1.46483641e+01,  1.49182236e+00,  2.58779602e+01],
       [-2.68746681e+01,  2.64852886e+01, -8.86370277e+00,
        -2.82666612e+00, -5.61382675e+01,  5.353875

In [22]:
d[290:299,90:99]

array([[-1.2312525e+01,  3.6738335e+01, -2.9995686e+01, -3.4591621e+01,
        -2.9205902e+01, -2.4409264e+01,  3.2718010e+00,  4.7244072e+01,
         3.3177925e+01],
       [ 7.3267031e+00,  5.6836128e-02, -4.1828618e+00, -2.3764481e+01,
        -3.2490257e+01,  3.2166679e+01, -1.2079239e+00, -9.6022148e+00,
         2.2822271e+01],
       [-2.8469173e+01, -2.1326492e+01,  1.0380196e+01, -4.6141052e+00,
        -2.4735275e+01,  8.2673168e+00,  1.0010129e+01, -7.7200680e+00,
        -5.7549934e+00],
       [ 9.7789621e+00, -3.2166466e+01,  1.4307817e+01, -3.7836269e+01,
         2.5611515e+00, -6.5604736e+01,  5.9657021e+00,  8.7400198e+00,
         5.6684837e+01],
       [-4.5023228e+01,  1.3602837e+01, -4.1208122e+01, -3.5254501e+01,
        -1.3182658e+01, -1.0014510e+01,  1.4648371e+01,  1.4918251e+00,
         2.5877937e+01],
       [-2.6874668e+01,  2.6485275e+01, -8.8637009e+00, -2.8266144e+00,
        -5.6138298e+01,  5.3538719e+01,  1.6939196e+01, -2.5861328e+01,
         3.

# Analysis
The Problem with the previous code is the memory access is no coalesced.<br>

![image.png](attachment:f31c927a-80fe-4bd4-90ac-074ff2f0f00a.png)


So, there is an algorithm which allows an smart memory handling, using the shared memory tiling: 

In [23]:
kernel_code_template = """
__global__ void MatrixMulKernel(float *A, float *B, float *C)
{

  const uint wA = %(MATRIX_A_COLS)s; //This is different because they are not square matrices
  const uint wB = %(MATRIX_B_COLS)s;  

  // Block index
  const uint bx = blockIdx.x;
  const uint by = blockIdx.y;

  // Thread index
  const uint tx = threadIdx.x;
  const uint ty = threadIdx.y;

  // Index of the first sub-matrix of A processed by the block
  const uint aBegin = wA * %(BLOCK_SIZE)s * by;
  // Index of the last sub-matrix of A processed by the block
  const uint aEnd = aBegin + wA - 1;
  // Step size used to iterate through the sub-matrices of A
  const uint aStep = %(BLOCK_SIZE)s;

  // Index of the first sub-matrix of B processed by the block
  const uint bBegin = %(BLOCK_SIZE)s * bx;
  // Step size used to iterate through the sub-matrices of B
  const uint bStep = %(BLOCK_SIZE)s * wB;

  // The element of the block sub-matrix that is computed
  // by the thread
  float Csub = 0;
  // Loop over all the sub-matrices of A and B required to
  // compute the block sub-matrix
  for (int a = aBegin, b = bBegin;
       a <= aEnd;
       a += aStep, b += bStep) 
    {
      // Shared memory for the sub-matrix of A
      __shared__ float As[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];
      // Shared memory for the sub-matrix of B
      __shared__ float Bs[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];

      // Load the matrices from global memory to shared memory
      // each thread loads one element of each matrix
      As[ty][tx] = A[a + wA * ty + tx];
      Bs[ty][tx] = B[b + wB * ty + tx];
      // Synchronize to make sure the matrices are loaded
      __syncthreads();

      // Multiply the two matrices together;
      // each thread computes one element
      // of the block sub-matrix
      for (int k = 0; k < %(BLOCK_SIZE)s; ++k)
        Csub += As[ty][k] * Bs[k][tx];

      // Synchronize to make sure that the preceding
      // computation is done before loading two new
      // sub-matrices of A and B in the next iteration
      __syncthreads();
    }

  // Write the block sub-matrix to global memory;
  // each thread writes one element
  const uint c = wB * %(BLOCK_SIZE)s * by + %(BLOCK_SIZE)s * bx;
  C[c + wB * ty + tx] = Csub;
}
"""


### Steps
Here, we define the number of rows and columns of both matrices. In this case, we have specified the dimension 1024x512 for the matrix A and 512x1024 for the matrix B.

In [58]:
rows_a=1024
cols_a=50
rows_b=50
cols_b=1024

We create matrix_a and matrix_b with its corresponding dimensions, filled with random numbers.

In [59]:
matrix_a=np.random.randn(rows_a,cols_a).astype(np.float32)
matrix_b=np.random.randn(rows_b,cols_b).astype(np.float32)
matrix_c=np.zeros((rows_a,cols_b),dtype=np.float32)

In [60]:
# define size of blocks and tiles sub-matrix 
# (we assume that the block size is same as tile size)
TILE_SIZE = 16
BLOCK_SIZE = TILE_SIZE

This is a trick, to replace in our original code the constants MATRIX_A_SIZE, MATRIX_B_SIZE and BLOCK_SIZE, for it values.

It allows us change the matriz sizes without change the original code.

It is not CUDA trick, its a python trick  using strings and % parameters.

In [61]:
# get the kernel code from the template 
# by specifying the constants MATRIX_A_SIZE, MATRIX_B_SIZE, and BLOCK_SIZE
#We call the number of columns of matrices A and B instead of the size of each matrix.
kernel_code = kernel_code_template % { 
    'MATRIX_A_COLS': cols_a,
    'MATRIX_B_COLS': cols_b,
    'BLOCK_SIZE': BLOCK_SIZE
}

Here,we compile the kernel code, geting the kernel function reference, in matrixmul.

In [62]:
# compile the kernel code
mod = SourceModule(kernel_code)

Then we have to crate our matrix_a and matrix_b to be used in this algorithm, and upload to the GPU, for this algorithm.

In [63]:
#We copy our matrices and upload them to de GPU for this algorithm
a_cpu = matrix_a
b_cpu = matrix_b

# compute reference on the CPU to verify GPU computation
c_cpu = np.dot(a_cpu, b_cpu)

Instead to create a host matrix c with zeroes, we can reserve the memory in the GPU, after we can copy the results from the memory to a new local variable.<br/>
This will reduce the amount of data to upload (and the time its taken).

In [64]:
# transfer host (CPU) memory to device (GPU) memory 
a_gpu = gpuarray.to_gpu(a_cpu) 
b_gpu = gpuarray.to_gpu(b_cpu)

# create empty gpu array for the result (C = A * B)
c_gpu = gpuarray.empty((rows_a, cols_b), np.float32)

In [65]:
# get the kernel function from the compiled module
matrixmul = mod.get_function("MatrixMulKernel")

Now, we define the blocksize and grid size parameters.

In [66]:
block_size=(int(TILE_SIZE),int(TILE_SIZE),1)

In [67]:
gris_size=(int(np.ceil(cols_a/TILE_SIZE)),int(np.ceil(cols_b/TILE_SIZE)))

Invokes the matrices multiplication kernel, and measure the timming.

In [68]:
# call the kernel on the card
start_t = time.time()
matrixmul(
    # inputs
    a_gpu, b_gpu, 
    # output
    c_gpu, 
    # grid of multiple blocks
    grid = grid_size,
    # block of multiple threads
    block = block_size 
    )
end_t = time.time()

In [69]:
print("Time processing: {0} seconds".format(end_t-start_t))

Time processing: 0.0003235340118408203 seconds


In [70]:
c_cpu_final = c_gpu.get()

In [71]:
c_cpu[0:5,0:5]

array([[-12.239344  ,  -9.442764  ,  -1.3886307 ,  -3.006447  ,
         -0.26099968],
       [ -2.9906766 ,  -9.16776   ,   0.18869138,   6.3902507 ,
          4.687619  ],
       [ -0.7491901 , -12.7215805 ,  12.837054  ,  -2.5493095 ,
        -11.776177  ],
       [  5.448393  ,   3.0563555 ,  -1.1583443 ,   0.73490596,
         -3.0179965 ],
       [-12.381034  ,  -8.46078   ,  -0.32425186,  -4.1947765 ,
          2.9934554 ]], dtype=float32)

In [72]:
c_cpu_final[0:5,0:5]

array([[-12.239345  ,  -9.442763  ,  -1.3886307 ,  -3.0064464 ,
         -0.26099902],
       [ -2.9906764 ,  -9.167759  ,   0.18869105,   6.3902507 ,
          4.687624  ],
       [ -0.74918866, -12.721581  ,  12.837055  ,  -2.5493097 ,
        -11.776179  ],
       [  5.448392  ,   3.0563555 ,  -1.1583446 ,   0.73490703,
         -3.0179973 ],
       [-12.381033  ,  -8.460781  ,  -0.3242505 ,  -4.1947765 ,
          2.9934556 ]], dtype=float32)

In [73]:
MSE = np.sum(np.sum(np.power(c_cpu_final-d,2)))/(d.shape[0]*d.shape[1])

In [74]:
print("Mean Square Error: {0}".format(MSE))

Mean Square Error: 1072.0140380859375


# PRACTICAL WORK:

Modify the previous code to recieive any dimensional matrices, and be able to multipy them using tiled memory.

Take care about boundaries in the final matrix, and fill with zeroes the memory places where the original matrices are not defined.

### Conclusions:

We can see that the error returned by the algorithm is higher when the difference between the number of rows and columns is bigger. For example, it returns a higher error when we run the algorithm with matrix A = 1024x50 and matrix B = 50x1024.